In [24]:
n=180
n1=60#control
n2=120
m=2000
p1=2#observed phenotype
p2=2#unobserved phenotype
K=3


X1=rbind(c(rep(0,n1),rep(1,n2)),runif(n,min=5,max=60))#(2,180),这两个性状一个是定性的，一个是定量的，下面也是
beta1=array(0,dim=c(m,K,p1))
#for beta1[,,1], assume it affects all three types at site 1:10, affects 1,2 at 11:20, affects 3 at 21:30
m_common <- 10
max_signal <- 0.5
min_signal <- 0.1
signs <- sample(c(-1,1), m_common*K, replace=TRUE)
beta1[1:m_common,1:K,1] <- signs * runif(m_common*K, min=min_signal, max=max_signal)

m_seperate <- 10
signs <- sample(c(-1,1), m_seperate*2, replace=TRUE)
beta1[m_common+(1:m_seperate),1:2,1] <- signs * 
					runif(m_seperate*2, min=min_signal, max=max_signal)

signs <- sample(c(-1,1), m_seperate, replace=TRUE)
beta1[m_common+m_seperate+(1:m_seperate),K,1] <- signs * 
					runif(m_seperate, min=min_signal, max=max_signal)
#for beta1[,,2], assume it affects all three types at site 50+1:10, affects 1,2 at 50+11:20, affects 3 at 50+21:30
#这里我们发现如果也是影响1：30的话后面会出很大的问题。。所以暂且加一个base=50，让他们影响点位不重合

base <- 0
m_common <- 10
max_signal <- 0.5
min_signal <- 0.1
signs <- sample(c(-1,1), m_common*K, replace=TRUE)
beta1[base+1:m_common,1:K,2] <- signs * 
					runif(m_common*K, min=min_signal, max=max_signal)

m_seperate <- 10
signs <- sample(c(-1,1), m_seperate*2, replace=TRUE)
beta1[base+m_common+(1:m_seperate),1:2,2] <- signs * 
					runif(m_seperate*2, min=min_signal, max=max_signal)

signs <- sample(c(-1,1), m_seperate, replace=TRUE)
beta1[base+m_common+m_seperate+(1:m_seperate),K,2] <- signs * 
					runif(m_seperate, min=min_signal, max=max_signal)

#unobserved phenotype
X2=rbind(c(rep(0,n1),rep(1,n2)),runif(n,min=5,max=60))#(2,180),这两个性状一个是定性的，一个是定量的，下面也是
beta2=array(0,dim=c(m,K,p2))
#for beta2[,,2], assume it affects all three types at site 1:10, affects 2,3 at 11:20, affects 1 at 21:30

m_common <- 10
max_signal <- 0.5
min_signal <- 0.1
signs <- sample(c(-1,1), m_common*K, replace=TRUE)
beta2[1:m_common,1:K,1] <- signs * runif(m_common*K, min=min_signal, max=max_signal)

m_seperate <- 10
signs <- sample(c(-1,1), m_seperate*2, replace=TRUE)
beta2[m_common+(1:m_seperate),2:3,1] <- signs * 
					runif(m_seperate*2, min=min_signal, max=max_signal)

signs <- sample(c(-1,1), m_seperate, replace=TRUE)
beta2[m_common+m_seperate+(1:m_seperate),1,1] <- signs * 
					runif(m_seperate, min=min_signal, max=max_signal)
#for beta2[,,2], assume it affects all three types at site 50+1:10, affects 1,2 at 50+11:30

base <- 0
m_common <- 10
max_signal <- 0.5
min_signal <- 0.1
signs <- sample(c(-1,1), m_common*K, replace=TRUE)
beta2[base+1:m_common,1:K,2] <- signs * 
					runif(m_common*K, min=min_signal, max=max_signal)

m_seperate <- 10
signs <- sample(c(-1,1), m_seperate*2, replace=TRUE)
beta2[base+m_common+(1:20),1:2,2] <- signs * 
					runif(m_seperate*2, min=min_signal, max=max_signal)

#composition
set.seed(123)
rDirichlet <- function(alpha_vec){
    num <- length(alpha_vec)
    temp <- rgamma(num, shape = alpha_vec, rate = 1)#shape 和rate是两个参数，不决定维度
    return(temp / sum(temp))
}
P <- sapply(1:n, function(i){
    if(X1[1,i]==0){ #if control
        rDirichlet(c(4,4, 2+X1[2,i]/10))
    }else{
        rDirichlet(c(4,4, 5+X1[2,i]/10))
    }
})#(3,180)



Ometh2 <- NULL
for(i in 1:n){
    utmp <- t(sapply(1:m, function(j){
        tmp1 <- colSums(X1[ ,i] * t(beta1[j, , ]))#beta(m,k,p), (2)*(2,3)
        rnorm(K,mean=mu[j, ]+tmp1,sd=0.01)#mu(2000,3)
    }))#这一部分，取定一个sample，按照paper里 o=mu+x*beta+epsilon的公式计算每一个点位的三种细胞的甲基化值.utmp(2000,3)
    dim(utmp)
    tmp2 <- colSums(P[ ,i] * t(utmp))#按P[3,180]中的构成比对每个sample取平均甲基化值
    Ometh2 <- cbind(Ometh2, tmp2 + rnorm(m, sd = 0.01))#测量结果有误差
}
Ometh2[Ometh2 > 1] <- 1
Ometh2[Ometh2 < 0] <- 0

In [25]:
head(Ometh2)

1.0000000,1.0000000,0,1,1.0000000,1,1.0000000,1,1,1,...,1,1.00000000,0,1.0000000,1,1,1,1,1,0.000000
1.0000000,1.0000000,1,1,1.0000000,1,1.0000000,1,1,1,...,1,1.00000000,1,1.0000000,1,1,1,1,1,1.000000
0.0000000,0.0000000,0,0,0.0000000,0,0.0000000,0,0,0,...,0,0.00000000,0,0.0000000,0,0,0,0,0,0.000000
1.0000000,1.0000000,0,1,1.0000000,1,1.0000000,1,1,1,...,1,1.00000000,0,1.0000000,1,1,1,1,1,0.222224
0.4863314,0.8184365,1,1,0.9904227,1,0.6127716,1,1,1,...,1,1.00000000,1,1.0000000,1,1,1,1,1,1.000000
1.0000000,0.8055681,0,0,0.3797455,0,1.0000000,0,0,0,...,0,0.03461331,0,0.1951495,0,0,0,0,0,0.000000


In [26]:
ret_list <- HIRE(Ometh, X, num_celltype=K, tol=10^(-5), num_iter=1000, alpha=0.01)


  Initialization Done.

  Implementing EM algorithm... 



Iteration: 1	 observed-data log likelihood: 784082.581817
Iteration: 2	 observed-data log likelihood: 841645.611960
Iteration: 3	 observed-data log likelihood: 884603.698583
Iteration: 4	 observed-data log likelihood: 920305.763266
Iteration: 5	 observed-data log likelihood: 950697.928674
Iteration: 6	 observed-data log likelihood: 976470.203803
Iteration: 7	 observed-data log likelihood: 997979.144202
Iteration: 8	 observed-data log likelihood: 1015556.285154
Iteration: 9	 observed-data log likelihood: 1029609.011855
Iteration: 10	 observed-data log likelihood: 1040628.736395
Iteration: 11	 observed-data log likelihood: 1049149.831921
Iteration: 12	 observed-data log likelihood: 1055704.829707
Iteration: 13	 observed-data log likelihood: 1060739.448801
Iteration: 14	 observed-data log likelihood: 1064647.720146
Iteration: 15	 observed-data log likelihood: 1067713.170403
Iteration: 16	 observed-data log likelihood: 1070129.974278
Iteration: 17	 observed-data log likelihood: 1072064.168

  Done! 

  Calculating p-values...

  Done!

